##### System Libraries

In [ ]:
import os
import subprocess
import logging
import re
import copy
import time
import math
import json
import pickle
import datetime
import decimal

##### Required Libraries

In [ ]:
try:
    import numpy as np
except ModuleNotFoundError:
    os.system('pip install numpy')
    import numpy as np
try:
    import scapy
except ModuleNotFoundError:
    os.system('pip install scapy')
    import scapy
try:
    import tqdm
except ModuleNotFoundError:
    os.system('pip install tqdm')
    import tqdm
try:
    import requests
except ModuleNotFoundError:
    os.system('pip install requests')
    import requests
try:
    import pandas as pd
except ModuleNotFoundError:
    os.system('pip install pandas')
    import pandas as pd
try:
    import sklearn
except ModuleNotFoundError:
    os.system('pip install scikit-learn')
    import sklearn
try:
    import matplotlib
except ModuleNotFoundError:
    os.system('pip install matplotlib')
    import matplotlib
    
logging.getLogger("requests").setLevel(logging.INFO)
logging.getLogger("urllib3").setLevel(logging.INFO)

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans,DBSCAN,AgglomerativeClustering,MeanShift,SpectralClustering,AffinityPropagation,Birch,OPTICS
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

from scapy.all import rdpcap
from tqdm import trange,tqdm

### Analyzer

#### Minor Function Breakdowns
---
##### Generate_Logger
Generates a logger with default parameters for logging critical components and debugging, default mode is INFO, for debugging change it to DEBUG
##### Generate Self IPs
Generates a list of IPs assigned to current device according to a protocol list. Uses 'ipconfig' from command prompt to get info.
##### List Initializer [V2]
Helper function to shorten and simplify initializing and re-initializing lists with values. Avoids creating lists within lists if used correctly.
##### Read JSON [V2] [DEPRECATED]
Reads a JSON file from the provided directory with the filename.(extension). Can work without the filename extension, will throw error if file doesn't exist.
##### Read List [V2] 
Reads a PKL file from the provided directory with the filename.(extension). Can work without the filename extension, will throw error if file doesn't exist.
##### Write JSON [V2] [DEPRECATED]
Creates a json file with the dictionary at the directory. Can work without filename extenstion.
##### Write List [V2]
Creates a pkl file with the dictionary at the directory. Can work without filename extenstion.
##### Remove Reserved IPs
Removes IPs that belong in the reserved ranges from the provided list. '192.168.' range is excluded.
##### Calc Trans time
Calculates the capture time of the pcap data
##### PCAP_Input [1.5]
Takes a PCAP file as input and stores it internally for later use
##### Generate Unique IPs [V2]
Generates a list of unique IPs found in the PCAP data (excluding self IPs), if already generated and saved, loads it. Can be forced.
##### Analyze Flow [V1.7]
##### Query IP
Queries the ip-api website for a single ip, waits for the response, and returns it back. Primarily a helper function.
##### Query IPs [V2]
Requests IP information from ip-api through requests, and stores the received data in a list, saves the list so we dont have to query every single time we run the program, saving time. Can be forced.
##### Generate Dataset [V1.2]
Should generate a dataset for use in clustering
##### Trace IPs
Traces the IPs and returns the output
##### Generate Summary [V1.1] [DEPRECATED]
Summarizes the data in a neater form from queried data for the user

Known Bugs

Will throw a 'Flow' KeyError if the queried_data and unique_ips dont match


Notes

IAT List must be in normal python float else numpy will throw error because it cannon handle Decimal data

if IP is assigned through DHCP, self_ip assignment and pcap_data self_ip may differ and show every field with default values

To-Do

-Empty-

In [ ]:

class Analyzer:
    if True:
        #Initialization of the logger, have to do it here, else it breaks stuff
        logger=logging.getLogger
        stored_logger_level=[logging.INFO]
        stored_self_ip_protocol_list=["IPv4","IPv6"]

        stored_pcap_data=[]
        stored_path=[]
        stored_directory=[]
        
        stored_transmission_time=[]
        
        stored_self_ips=[]
        stored_unique_ips=[]
        stored_blocked_ips=[]

        stored_queried_data=[]
        stored_flow=[]
        stored_protocol_dict={'1':'ICMP','2':'IGMP','3':'GGP','4':'IPv4','5':'Stream','6':'TCP','17':'UDP'}
        stored_port_dict={'53':'DNS','80':'HTTP','137':'NetBIOS','443':'HTTPS','1900':'SSDP','4070':'Amzn EDot2Sptfy','5222':'XMPP','22101':'GImpact','23301':'HSR'}
        
        stored_df=[pd.DataFrame()]
        
    def __init__(self,path,protocol_list=stored_self_ip_protocol_list,features=['Burst Down','Burst Up'],
                 input=True,remove_reserved=True,
                 query=True,forced_query=False,infinite_query=False,query_amount=45,
                 replace=True,ping=True,cluster=True,clustering_algorithm='kmeans',clusters=6,samples=2,
                 forced=False,verbose=True):
        if True:
            self.logger=self.Generate_Logger(filename='AnalysisCore.log')
            logging.getLogger("requests").setLevel(logging.INFO)
            np.seterr(invalid='ignore')
            self.Generate_Self_IPs(protocol_list=protocol_list)
            directory=path[:path.find('.')]
            self.List_Initializer(self.stored_path,path)
            self.List_Initializer(self.stored_directory,directory)
            if input:
                self.PCAP_Input(path)
            self.Calc_Trans_Time(forced=forced,verbose=verbose)
            self.Generate_Unique_IPs(remove_reserved=remove_reserved,forced=forced,verbose=verbose)
            self.Analyze_Flow(forced=forced,verbose=verbose)
            if query==True:
                self.Query_IPs(amount=query_amount,forced=forced_query,verbose=verbose)
                if infinite_query:
                    while len(self.stored_unique_ips)>len(self.stored_queried_data):
                        self.Query_IPs(amount=query_amount,verbose=verbose)
            self.Generate_Dataset(forced=forced,verbose=verbose,replace=replace,ping=ping)
            if cluster:
                self.Generate_Cluster(features=features,clustering_algorithm=clustering_algorithm,clusters=clusters,samples=samples)
    def Generate_Logger(self,filename='default_logger_name.log'): 
        logging.basicConfig(filename=filename,format='%(asctime)s - %(levelname)s: %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',filemode='w',level=self.stored_logger_level[0])
        logger=logging.getLogger()
        logger.debug('Logger initialized.')
        return logger
    def Generate_Self_IPs(self,protocol_list=stored_self_ip_protocol_list):
        self.logger.info("Generating self IPs with protocols, "+str(protocol_list))
        self_ips=[]
        process=subprocess.Popen(['ipconfig'],stdout=subprocess.PIPE,text=True,shell=True)
        for line in process.stdout:
            for protocol in protocol_list:
                if re.search(protocol,line.strip()):
                    self_ips.append(line.strip()[line.strip().find(':')+2:])
        self.List_Initializer(self.stored_self_ips,self_ips)
        self.logger.debug('Stored self IPs in stored_self_ips '+str(self_ips))
        return self_ips
    def List_Initializer(self,input_list,input_param):
        if isinstance(input_param,list) or isinstance(input_param,scapy.plist.PacketList):
            input_list.clear()
            for item in input_param:
                input_list.append(item)
        else:
            input_list.clear()
            input_list.append(input_param)
    def Dec_Trunc(self,input_decimal,decimal=100000):
        return math.trunc(input_decimal*decimal)/decimal

    def Read_JSON(self,directory,filename): #deprecated
        name, ext=os.path.splitext(filename)
        if os.path.exists(f'{directory}/{name}.json'): #if the file exists,
            self.logger.debug(f'Reading JSON from {directory}/{filename}') 
            with open(f'{directory}/{name}.json') as file: #then open it
                data=json.load(file) #save it
                return data #return it
        else:
            self.logger.error(f'Trying to open {filename} as JSON')
    def Read_List(self,directory,filename): 
        name, ext=os.path.splitext(filename)
        if os.path.exists(f'{directory}/{name}.pkl'): #if the file exists,
            self.logger.debug(f'Reading PKL from {directory}/{filename}')
            with open(f'{directory}/{name}.pkl','rb') as file: #then open it
                data=pickle.load(file) #save it
                return data #return it
        else:
            self.logger.error(f'Trying to open {filename} as PKL')
    def Write_JSON(self,dictionary,directory,filename): #deprecated
        name, ext=os.path.splitext(filename)
        if ext!='.json': #make sure we are writing a json
            if ext!='':
                self.logger.error(f'Trying to write {filename} as {ext} is not JSON')
                return
        os.makedirs(directory,exist_ok=True) #make sure the directory exists, if it does not, create it
        self.logger.debug(f'Writing JSON at {directory}/{name}.json')
        with open(f'{directory}/{name.replace(':','')}.json','w') as file:  #create a file
            json.dump(dictionary,file,indent=4) #dump the dictionary in the file
    def Write_List(self,list,directory,filename):
        name, ext=os.path.splitext(filename)
        if ext!='.pkl': #make sure we are writing a pkl
            if ext!='':
                self.logger.error(f'Trying to write {filename} as {ext} is not PKL')
                return
        os.makedirs(directory,exist_ok=True) #make sure the directory exists, if it does not, create it
        self.logger.debug(f'Writing PKL at {directory}/{name}.pkl')
        with open(f'{directory}/{name.replace(':','')}.pkl','wb') as file:  #create a file
            pickle.dump(list,file) #dump the list in the file

    def Remove_Reserved_IPs(self,ips):
        self.logger.debug(f'Removing reserved IP ranges from list')
        dot=re.escape('.')
        patterns=[f'0{dot}0{dot}0{dot}0', f'10{dot}', f'127{dot}', f'169{dot}254{dot}', f'192{dot}0{dot}0{dot}', f'192{dot}0{dot}2{dot}', f'192{dot}18{dot}', f'192{dot}19{dot}',f'192{dot}51{dot}100{dot}', f'192{dot}88{dot}99{dot}', f'203{dot}0{dot}113{dot}', f'255{dot}', f'256{dot}']+[f'100{dot}{x}{dot}' for x in range(64,128)]+[f'172{dot}{x}{dot}' for x in range(16,32)]+[f'{x}{dot}' for x in range(224,240)]
        #self.logger.debug(patterns)
        temp=[]
        for ip in ips:
            for pattern in patterns:
                if re.match(pattern,ip):
                    self.logger.debug(f'Pruning {ip} with {pattern}')
                    temp.append(ip)
        for ip in ips:
            if re.search('::',ip):
                self.logger.debug(f'Pruning IPv6 IP {ip}')
                temp.append(ip)
        for temp_ip in temp:
            ips.remove(temp_ip)
        self.logger.debug(f'Removed {temp}')
        return ips        
    def Calc_Trans_Time(self,pcap_data=stored_pcap_data,forced=False,verbose=True):
        time=[]
        directory=self.stored_directory[0]
        filename='time.pkl'
        data_not_found=False
        if not forced: #load existing data if found
            if os.path.exists(f'{directory}/{filename}'): #if existing data is found
                self.logger.info(f'Found existing time data at {directory}/{filename}')
                read_list=self.Read_List(directory,filename)
                self.List_Initializer(self.stored_unique_ips,read_list)
                if verbose:
                    print(f'Found time data. [{read_list}]')
                return read_list
            else:
                data_not_found=True
        if (forced or data_not_found): #generate data
            self.logger.debug(f'Calculating transmission time.')
            time=math.floor(pcap_data[-1].time-pcap_data[0].time)
            self.Write_List(time,directory,filename)
            self.List_Initializer(self.stored_transmission_time,time)
            return time

    def PCAP_Input(self,path):

        file_size=math.ceil(os.path.getsize(path)/math.pow(1024,2))
        print(f'Reading PCAP Data. [{file_size} MB]')
        self.logger.info(f'Reading PCAP data [{file_size} MB] from path, "{path}"')
        pcap_data=rdpcap(path)
        self.List_Initializer(self.stored_pcap_data,pcap_data)
        self.logger.debug(f'Stored {path} data in stored_pcap_data.')
        return pcap_data

    def Generate_Unique_IPs(self,pcap_data=stored_pcap_data,self_ips=stored_self_ips,remove_reserved=True,verbose=True,forced=False,save=True):
        unique_ips=[]
        directory=self.stored_directory[0]
        filename='unique_ips.pkl'
        data_not_found=False
        if not forced: #load existing data if found
            if os.path.exists(f'{directory}/{filename}'): #if existing data is found
                self.logger.info(f'Found existing unique_ips data at {directory}/{filename}')
                read_list=self.Read_List(directory,filename)
                self.List_Initializer(self.stored_unique_ips,read_list)
                if verbose:
                    print(f'Found unique_ips data. [{len(read_list)}]')
                return read_list
            else:
                data_not_found=True
        if (forced or data_not_found): #generate data
            self.logger.info('Extracting Unique IPs from PCAP Data.')
            blocked_ips=self_ips.copy()
            packets=pcap_data
            if verbose:
                for packet in tqdm(packets,desc='Analyzing packets for unique IPs'):
                    flag=0
                    try:
                        for ip in blocked_ips:
                            if packet.payload.dst==ip:
                                flag=1
                        if flag==0:
                            blocked_ips.append(packet.payload.dst)
                            unique_ips.append(packet.payload.dst)
                    except AttributeError:
                        broadcast_temp='256.256.256.256'
                        for ip in blocked_ips:
                            if ip==broadcast_temp:
                                flag=1
                        if flag==0:
                            blocked_ips.append(broadcast_temp)
                            unique_ips.append(broadcast_temp)
            else:
                for packet in packets:
                    flag=0
                    try:
                        for ip in blocked_ips:
                            if packet.payload.dst==ip:
                                flag=1
                        if flag==0:
                            blocked_ips.append(packet.payload.dst)
                            unique_ips.append(packet.payload.dst)
                    except AttributeError:
                        broadcast_temp='256.256.256.256'
                        for ip in blocked_ips:
                            if ip==broadcast_temp:
                                flag=1
                        if flag==0:
                            blocked_ips.append(broadcast_temp)
                            unique_ips.append(broadcast_temp)
            if remove_reserved==True:
                self.Remove_Reserved_IPs(unique_ips)
            if save==True:
                self.Write_List(unique_ips,directory,'unique_ips')
            self.List_Initializer(self.stored_unique_ips,unique_ips)
        self.logger.debug('Stored unique IPs in stored_unique_ips.')
        return unique_ips
    def Analyze_Flow(self,pcap_data=stored_pcap_data,forced=False,verbose=True):
        directory=self.stored_directory[0]
        filename='flow.pkl'
        data_not_found=False
        
        if not forced: #load existing data if found
            if os.path.exists(f'{directory}/{filename}'): #if existing data is found
                self.logger.info(f'Found existing flow data at {directory}/{filename}')
                read_list=self.Read_List(directory,filename)
                self.List_Initializer(self.stored_flow,read_list)
                if verbose:
                    print(f'Found analyzed flow data. [{len(read_list)}]')
                return read_list
            else:
                data_not_found=True
        if (forced or data_not_found):
            firstmost_pkt_time=pcap_data[0].time
            analyzed_flow=[]
            trans_time=self.stored_transmission_time[0]
            self.logger.info(f'Calculating flow [{len(pcap_data)}]')
            if verbose:
                print(f'Analyzing packets. [{len(pcap_data)}]')
            unique_ips=copy.deepcopy(self.stored_unique_ips)
            for ip in self.Remove_Reserved_IPs(unique_ips): #initialize with all the unique ips except self ips
                analyzed_flow.append([ip,0,0,0,0,[],[],[],(-1),0,0,0,[0],[0],[0]]) 
                #IP, Down Bytes, Up Bytes, Down Packets, Up Packets, Used Protocols, S Ports, D Ports, First Packet Time, Last Mixed Packet Time, Last Dl Pkt Time, Last Up Pkt Time, IAT Mixed, IAT Down, IAT Up
            
            for packet in pcap_data: #for every packet, the analysis of the flow of data happens in this loop
                for self_ip in self.stored_self_ips: #for every self ip
                    try:
                        if packet.payload.dst==self_ip: #check if payload destination is self ip (download)
                            for flow_data in analyzed_flow: #for every unique ip tuple
                                try:
                                    ip=flow_data[0]
                                    if ip==packet.payload.src: #if the payload source matches with the unique ip
                                        if True: #variable assignment for better clarification and assignment
                                            downloaded_bytes=flow_data[1]
                                            uploaded_bytes=flow_data[2]
                                            downloaded_packets=flow_data[3]
                                            uploaded_packets=flow_data[4]
                                            protocol_list=flow_data[5]
                                            source_ports=flow_data[6]
                                            destination_ports=flow_data[7]
                                            first_packet_time=flow_data[8]
                                            last_mixed_packet_time=flow_data[9]
                                            last_downloaded_packet_time=flow_data[10]
                                            last_uploaded_packet_time=flow_data[11]
                                            iat_mixed=flow_data[12]
                                            iat_download=flow_data[13]
                                            iat_upload=flow_data[14]
                                        downloaded_bytes+=packet.payload.len #add the packet length/size to stored downloaded bytes, payload size is 14 less than the packet size
                                        downloaded_packets+=1 #count the number of packets downloaded
                                        if packet.payload.proto not in protocol_list: #if the protocol is not already in the list, add it to the list
                                            protocol_list.append(packet.payload.proto)
                                        if packet.payload.sport not in source_ports: #if the source port is not already in the list, add it to the list
                                            source_ports.append(packet.payload.sport)
                                        if packet.payload.dport not in destination_ports: #if the destination port is not already in the list, add it to the list
                                            destination_ports.append(packet.payload.dport)
                                        if first_packet_time==(-1): #find the first packet time
                                            first_packet_time=packet.time-firstmost_pkt_time
                                        if (iat_download==[0] and downloaded_packets<=1): #if IAT Download list is empty and it is the first packet, pass the first time and note the time, so the first entry is zero, then start
                                            last_downloaded_packet_time=packet.time-firstmost_pkt_time #note the downloaded packet's relative time
                                        else:
                                            iat_download.append(float(packet.time-firstmost_pkt_time-last_downloaded_packet_time)) #calculate the IAT of the following mixed packets
                                            last_downloaded_packet_time=packet.time-firstmost_pkt_time #keep renoting the mixed packet's relative time
                                        if (iat_mixed==[0] and downloaded_packets<=1): #if IAT Mixed list is empty and it is the first packet, pass the first time and note the time, so the first entry is zero, then start
                                            last_mixed_packet_time=packet.time-firstmost_pkt_time #note the mixed packet's relative time
                                        else:
                                            iat_mixed.append(float(packet.time-firstmost_pkt_time-last_mixed_packet_time)) #calculate the IAT of the following mixed packets
                                            last_mixed_packet_time=packet.time-firstmost_pkt_time #keep renoting the mixed packet's relative time
                                        if True: #the non-list variables must be re-assigned here cause they are non-mutable
                                            flow_data[1]=downloaded_bytes
                                            flow_data[2]=uploaded_bytes
                                            flow_data[3]=downloaded_packets
                                            flow_data[4]=uploaded_packets
                                            flow_data[8]=first_packet_time
                                            flow_data[9]=last_mixed_packet_time
                                            flow_data[10]=last_downloaded_packet_time
                                            flow_data[11]=last_uploaded_packet_time
                                        break
                                except AttributeError:
                                    pass
                            break
                        elif packet.payload.src==self_ip: #if the destination isnt a self ip, then it must be upload
                            for flow_data in analyzed_flow: #for every unique ip
                                try:
                                    ip=flow_data[0]
                                    if ip==packet.payload.dst: #if the payload destination matches with the unique ip
                                        if True: #variable assignment for better clarification and assignment
                                            downloaded_bytes=flow_data[1]
                                            uploaded_bytes=flow_data[2]
                                            downloaded_packets=flow_data[3]
                                            uploaded_packets=flow_data[4]
                                            protocol_list=flow_data[5]
                                            source_ports=flow_data[6]
                                            destination_ports=flow_data[7]
                                            first_packet_time=flow_data[8]
                                            last_mixed_packet_time=flow_data[9]
                                            last_downloaded_packet_time=flow_data[10]
                                            last_uploaded_packet_time=flow_data[11]
                                            iat_mixed=flow_data[12]
                                            iat_download=flow_data[13]
                                            iat_upload=flow_data[14]
                                        uploaded_bytes+=packet.payload.len #add the packet length/size to stored uploaded bytes, payload size is 14 less than the packet size
                                        uploaded_packets+=1 #count the number of packets uploaded
                                        if packet.payload.proto not in protocol_list: #if the protocol is not already in the list, add it to the list
                                            protocol_list.append(packet.payload.proto)
                                        if packet.payload.sport not in source_ports: #if the source port is not already in the list, add it to the list
                                            source_ports.append(packet.payload.sport)
                                        if packet.payload.dport not in destination_ports: #if the destination port is not already in the list, add it to the list
                                            destination_ports.append(packet.payload.dport)
                                        if first_packet_time==(-1): #find the first packet time
                                            first_packet_time=packet.time-firstmost_pkt_time
                                        if (packet.time-firstmost_pkt_time) > last_mixed_packet_time: #find the last mixed packet time
                                            last_mixed_packet_time=packet.time-firstmost_pkt_time
                                        if (iat_upload==[0] and uploaded_packets<=1): #if IAT Upload list is empty and it is the first packet, pass the first time and note the time, so the first entry is zero, then start
                                            last_uploaded_packet_time=packet.time-firstmost_pkt_time #note the uploaded packet's relative time
                                        else:
                                            iat_upload.append(float(packet.time-firstmost_pkt_time-last_uploaded_packet_time)) #calculate the IAT of the following uploaded packets
                                            last_uploaded_packet_time=packet.time-firstmost_pkt_time #keep renoting the uploaded packet's relative time
                                        if (iat_mixed==[0] and uploaded_packets<=1): #if IAT Mixed list is empty and it is the first packet, pass the first time and note the time, so the first entry is zero, then start
                                            last_mixed_packet_time=packet.time-firstmost_pkt_time #note the mixed packet's relative time
                                        else:
                                            iat_mixed.append(float(packet.time-firstmost_pkt_time-last_mixed_packet_time)) #calculate the IAT of the following mixed packets
                                            last_mixed_packet_time=packet.time-firstmost_pkt_time #keep renoting the mixed packet's relative time
                                        if True: #the non-list variables must be re-assigned here cause they are non-mutable
                                            flow_data[1]=downloaded_bytes
                                            flow_data[2]=uploaded_bytes
                                            flow_data[3]=downloaded_packets
                                            flow_data[4]=uploaded_packets
                                            flow_data[8]=first_packet_time
                                            flow_data[9]=last_mixed_packet_time
                                            flow_data[10]=last_downloaded_packet_time
                                            flow_data[11]=last_uploaded_packet_time
                                        break
                                except AttributeError: #if removed, broadcast packets will throw errors
                                    pass
                    except AttributeError:
                        pass
            for flow_data in analyzed_flow: #2nd iteration of analysis happens here
                dl_bytes=flow_data[1]
                up_bytes=flow_data[2]
                dl_pkts=flow_data[3]
                up_pkts=flow_data[4]
                first_pkt_time=flow_data[8]
                last_pkt_time=flow_data[9]
                iats_mixed=flow_data[12]
                iats_down=flow_data[13]
                iats_up=flow_data[14]
                flow_data.append(self.Dec_Trunc(dl_bytes/trans_time,1000)) #avg download speed in bytes/s
                flow_data.append(self.Dec_Trunc(up_bytes/trans_time,1000)) #avg upload speed in bytes/s
                try:
                    down_psize=self.Dec_Trunc(dl_bytes/dl_pkts,1000)
                except ZeroDivisionError:
                    down_psize=0
                try:
                    up_psize=self.Dec_Trunc(up_bytes/up_pkts,1000)
                except ZeroDivisionError:
                    up_psize=0
                flow_duration=self.Dec_Trunc(last_pkt_time-first_pkt_time,1000)
                try:
                    flow_rate=self.Dec_Trunc((dl_bytes+up_bytes)/flow_duration,1000)
                except ZeroDivisionError:
                    flow_rate=0
                burstiness_mixed=np.std(iats_mixed)/np.mean(iats_mixed)
                burstiness_down=np.std(iats_down)/np.mean(iats_down)
                burstiness_up=np.std(iats_up)/np.mean(iats_up)
                if np.isnan(burstiness_mixed):
                    burstiness_mixed=np.float64(0)
                if np.isnan(burstiness_down):
                    burstiness_down=np.float64(0)
                if np.isnan(burstiness_up):
                    burstiness_up=np.float64(0)
                flow_data.append(burstiness_mixed)
                flow_data.append(burstiness_down)
                flow_data.append(burstiness_up)
                flow_data.append(flow_duration)
                flow_data.append(flow_rate)
                flow_data.append(down_psize)
                flow_data.append(up_psize)
            self.List_Initializer(self.stored_flow,analyzed_flow)
            directory=self.stored_directory[0]
            self.Write_List(analyzed_flow,directory,filename)
        self.logger.debug('Stored total flow in stored_flow')
        return analyzed_flow
    
    def Query_IP(self,ip):
        self.logger.debug(f'Querying for IP {ip}')
        response=requests.get(f"http://ip-api.com/json/{ip}")
        data=response.json()
        return data    
    def Query_IPs(self,ips=stored_unique_ips,amount=45,forced=False,verbose=True):
        queried_data=[]
        to_be_queried=ips.copy()
        directory=self.stored_directory[0]
        filename='queried_data.pkl'
        incomplete=False
        data_not_found=False
        if not forced: #load existing data if found
            if os.path.exists(f'{directory}/{filename}'): #if existing data is found
                self.logger.info(f'Found existing query data at {directory}/{filename}')
                read_list=self.Read_List(directory,filename)
                self.List_Initializer(queried_data,read_list)
                if len(read_list)<len(to_be_queried):
                    incomplete=True
                if verbose:
                    print(f'Found query data. [{len(queried_data)}]')
                    if incomplete:
                        print(f'Query data is incomplete. [{min(len(to_be_queried)-len(read_list),amount)}+{max(0,len(to_be_queried)-len(read_list)-amount)}]')
            else:
                data_not_found=True
        if (forced or incomplete or data_not_found):
            if incomplete: #if incomplete, prune
                self.logger.debug(f'Pruning IPs')
                prune=[]
                for query in queried_data: #for every queried IP
                    for ip in to_be_queried: #for every IP to be queried
                        if query[0]==ip: #check if the IP has already been queried
                            prune.append(ip) #add the IP to a pruning list
                            break
                for pruning_ip in prune: #prune the IPs
                    to_be_queried.remove(pruning_ip)
            self.logger.info(f'Querying IPs [{len(to_be_queried)}]')
            if verbose:
                for ip in tqdm(to_be_queried[:min(amount,45)],desc='Querying IPs'):
                    response=self.Query_IP(ip)
                    temp=[ip]
                    temp.append(response)
                    queried_data.append(temp)
            else:
                print(f'Query_IPs verbose=false incomplete')
            self.Write_List(queried_data,directory,filename)
        self.List_Initializer(self.stored_queried_data,queried_data)
        self.logger.debug('Stored queried data in stored_queried_data')
        
        return queried_data
    

        if self.stored_flow!=[]: #if there is flow data, add it to summary
            for summary in summarized:
                for flow_data in self.stored_flow:
                    if flow_data[0]==summary['IP']:
                        try:
                            down_psize=math.trunc(flow_data[1]/flow_data[3]*100)/100
                        except ZeroDivisionError:
                            down_psize=0
                        try:
                            up_psize=math.trunc(flow_data[2]/flow_data[4]*100)/100
                        except ZeroDivisionError:
                            up_psize=0
                        summary['Flow']=f'{flow_data[5]} byte/s Down, {flow_data[6]} byte/s Up. Packets, {flow_data[3]} Received, {flow_data[4]} Sent'
                        summary['Misc']=f'Average Packet Size {down_psize} Down, {up_psize} Up'
                            
        return summarized

    def Generate_Dataset(self,data_in=stored_flow,replace=True,ping=True,forced=False,verbose=True):
        directory=self.stored_directory[0]
        filename='dataframe.csv'
        data_not_found=False
        data=copy.deepcopy(data_in)
        if not forced: #load existing data if found
            if os.path.exists(f'{directory}/{filename}'): #if existing data is found
                self.logger.info(f'Found existing dataframe at {directory}/{filename}')
                df=pd.read_csv(f'{directory}/{filename}')
                if verbose:
                    print(f'Found dataframe. [{len(df)}]')
            else:
                data_not_found=True
        if (forced or data_not_found):
            protocols=self.stored_protocol_dict
            ports=self.stored_port_dict
            self.logger.info(f'Generating dataset')
            if verbose:
                print(f'Generating Dataset [{len(data)}]')
            if replace:
                for flow_data in data:
                    protocol_data=flow_data[5]
                    sport_data=flow_data[6]
                    sport_prune=[]
                    dport_data=flow_data[7]
                    dport_prune=[]
                    for protocol in protocol_data:
                        if str(protocol) in protocols:
                            protocol_data[protocol_data.index(protocol)]=protocols.get(str(protocol))
                        else:
                            self.logger.error(f'Protocol {protocol} not found in dictionary')
                    for sport in sport_data:
                        if str(sport) in ports:
                            sport_data[sport_data.index(sport)]=ports.get(str(sport))
                        elif sport>=49152 and sport<=65535: #all the dynamic ports
                            if 'Dynamic' not in sport_data: #if a dynamic port doesnt already exist, make one, rest go in garbage
                                sport_data[sport_data.index(sport)]='Dynamic'
                            else:
                                sport_prune.append(sport) #add all the dynamic ports to a pruning list
                        elif sport>=27000 and sport<=27038: #steam ports
                            if 'Steam' not in sport_data:
                                sport_data[sport_data.index(sport)]='Steam'
                            else:
                                sport_prune.append(sport)
                        else:
                            self.logger.debug(f'Port {sport} not found in dictionary')
                    for dport in dport_data:
                        if str(dport) in ports:
                            dport_data[dport_data.index(dport)]=ports.get(str(dport))
                        elif dport>=49152 and dport<=65535: #all the dynamic ports
                            if 'Dynamic' not in dport_data: #if a dynamic port doesnt already exist, make one, rest go in garbage
                                dport_data[dport_data.index(dport)]='Dynamic'
                            else:
                                dport_prune.append(dport) #add all the dynamic ports to a pruning list
                        elif dport>=27000 and dport<=27038: #steam ports
                            if 'Steam' not in dport_data:
                                dport_data[dport_data.index(dport)]='Steam'
                            else:
                                sport_prune.append(dport)
                        else:
                            self.logger.debug(f'Port {dport} not found in dictionary')
                    for sport in sport_prune: #prune the ports
                        sport_data.remove(sport)
                    for dport in dport_prune: #prune the ports
                        dport_data.remove(dport)
            df=pd.DataFrame(data,columns=['IP Address','Bytes Down','Bytes Up','Pkts Down','Pkts Up','Protocol(s)','Source Ports','Destination Ports',
                                          'First Pkt Time','Last Down Pkt Time','Last Up Pkt Time','Last Mixed Pkt Time',
                                          'IAT Mixed','IAT Down','IAT Up',
                                          'Avg Down B/s','Avg Up B/s',
                                          'Burst Mixed','Burst Down','Burst Up',
                                          'Flow Dura','Flow Rate',
                                          'Avg Down PSize','Avg Up PSize'])
            if self.stored_queried_data!=[]:
                to_be_appended=[]
                for query_data in self.stored_queried_data:
                    temp={}
                    query=query_data[1]
                    if query['status']=='success':
                        temp['Organization']=f"{query['as']}, {query['org']}, {query['isp']}"
                        temp['Location']=f"{query['city']}, {query['regionName']}, {query['country']} | {query['lat']}, {query['lon']}"
                    elif query['status']=='fail':
                        temp['Location']=f'Unknown'
                        temp['Organization']=f'Unknown'
                    to_be_appended.append(temp)
                q_df=pd.DataFrame(to_be_appended)
                df=pd.concat([df,q_df],axis=1)
            if ping:
                count=10
                timeout=1000
                packet_size=32
                processes=[]
                outputs=[]
                if verbose:
                    print(f'Pinging IPs [{len(data)}]')
                for flow_data in data:
                    ip=flow_data[0]
                    processes.append(subprocess.Popen(["ping",ip,"-n",str(count),"-w",str(timeout),"-l",str(packet_size)],
                                                      stdout=subprocess.PIPE,text=True,shell=True))
                time.sleep(min(int(count*(timeout/1000)*(1+len(data)*0.001)),int(count*5)))
                for process in processes:
                    outputs.append(process.stdout)
                ping_data=[]
                for output in outputs:
                    loss="0%"
                    avg="N/A"
                    for line in output:
                        string=line.strip()
                        if re.search('loss',string):
                            loss=string[string.find('(')+1:string.find(')')-5]
                        if re.search('Destination host unreachable',string):
                            avg="DHU"
                        elif re.search('Request timed out',string):
                            avg="RTO"
                        elif re.search('General failure',string):
                            avg="GF"
                        elif re.search('Average',string):
                            avg=string[string.find('Average = ')+10:]
                    temp=[avg,loss]
                    ping_data.append(temp)
                ping_df=pd.DataFrame(ping_data,columns=['Ping','Loss'])
                df=pd.concat([df,ping_df],axis=1)
            
            df.to_csv(f'{directory}/{filename}',index=False)
        self.List_Initializer(self.stored_df,df)
        self.logger.debug(f'Stored dataframe in stored_dataframe')
        return df
        
    def Generate_Cluster(self,df_input=stored_df,features=['Burst Down','Burst Up'],clustering_algorithm='kmeans',clusters=6,samples=2,show=True):
        df=df_input[0]
        scaler=StandardScaler()
        features_scaled=scaler.fit_transform(df[features])
        if show:
            self.logger.info(f'Generating cluster using {clustering_algorithm}')
        match clustering_algorithm:
            case 'kmeans':
                kmeans=KMeans(n_clusters=clusters)
                prediction=kmeans.fit_predict(features_scaled)
                df[f'Cluster [{clustering_algorithm}]']=prediction
            case 'dbscan':
                dbscan=DBSCAN(eps=0.5,min_samples=samples)
                prediction=dbscan.fit_predict(features_scaled)
                df[f'Cluster [{clustering_algorithm}]']=prediction
            case 'agglomerative':
                agglo=AgglomerativeClustering(n_clusters=clusters)
                prediction=agglo.fit_predict(features_scaled)
                df[f'Cluster [{clustering_algorithm}]']=prediction
            case 'meanshift':
                meanshift=MeanShift()
                prediction=meanshift.fit_predict(features_scaled)
                df[f'Cluster [{clustering_algorithm}]']=prediction
            case 'spectral':
                spectral=SpectralClustering(n_clusters=clusters, affinity='nearest_neighbors')
                prediction=spectral.fit_predict(features_scaled)
                df[f'Cluster [{clustering_algorithm}]']=prediction
            case 'affinity':
                affinity=AffinityPropagation()
                prediction=affinity.fit_predict(features_scaled)
                df[f'Cluster [{clustering_algorithm}]']=prediction
            case 'birch':
                birch=Birch(n_clusters=clusters)
                prediction=birch.fit_predict(features_scaled)
                df[f'Cluster [{clustering_algorithm}]']=prediction
            case 'optics':
                optics=OPTICS(min_samples=samples, xi=0.05, min_cluster_size=0.1)
                prediction=optics.fit_predict(features_scaled)
                df[f'Cluster [{clustering_algorithm}]']=prediction
                
        if show:
            plt.figure(figsize=(8, 6))
            scatter=plt.scatter(df[features[0]],df[features[1]],
                                c=df[f'Cluster [{clustering_algorithm}]'],cmap='viridis',s=100,edgecolor='k')
            plt.xlabel(f'{features[0]}')
            plt.ylabel(f'{features[1]}')
            plt.title(f'Network Flow Clustering by {features[0]} and {features[1]}')
            plt.colorbar(scatter, label=f'Cluster [{clustering_algorithm}]')
            plt.grid(True)
            plt.show()
            return
        else:
            return prediction
    def Generate_Ensembling(self,df_input=stored_df,features=['Burst Down','Burst Up'],ensembling_algorithms=['kmeans','dbscan','agglomerative'],clusters=6,samples=2,show=True):
        df=df_input[0]
        clusterings=[]
        self.logger.info(f'Generating cluster ensemble using {ensembling_algorithms}')
        for algo in ensembling_algorithms:
            clusterings.append(self.Generate_Cluster(features=features,clustering_algorithm=algo,clusters=clusters,samples=samples,show=False))
        
        n_samples=len(df)
        n_clusterings=len(clusterings)
        coassoc_matrix=np.zeros((n_samples,n_samples))
        
        for labels in clusterings:
            for i in range(n_samples):
                for j in range(n_samples):
                    if labels[i] == labels[j]:
                        coassoc_matrix[i][j] += 1
        
        coassoc_matrix/=n_clusterings
        dissimilarity_matrix=1-coassoc_matrix
        consensus_labels=AgglomerativeClustering(n_clusters=clusters,metric='precomputed',linkage='average').fit_predict(dissimilarity_matrix)  #Dissimilarity= 1-Similarity
        df[f'Cluster [CSPA]']=consensus_labels
        result=[]
        temp=[]
        count=0
        for algo in clusterings:
            ari=adjusted_rand_score(clusterings[count],consensus_labels)
            nmi=normalized_mutual_info_score(clusterings[count],consensus_labels)
            temp=[]
            temp.append(ensembling_algorithms[count])
            temp.append(ari)
            temp.append(nmi)
            result.append(temp)
            count+=1
        if show:
            plt.figure(figsize=(8, 6))
            scatter=plt.scatter(df[features[0]],df[features[1]],
                                c=consensus_labels,cmap='viridis',s=100,edgecolor='k')
            plt.xlabel(f'{features[0]}')
            plt.ylabel(f'{features[1]}')
            plt.title(f'Network Flow Clustering by {features[0]} and {features[1]}')
            plt.colorbar(scatter, label=f'Cluster [CSPA]')
            plt.grid(True)
            plt.show()
            return
        return result
    def Trace_IPs(self,ips=stored_unique_ips,amount=99,hops=20,timeout=1000):
        self.logger.info(f'Tracing IPs {ips[:amount]}')
        processes=[]
        for ip in ips[:amount]:
            processes.append(subprocess.Popen(["tracert","-h",str(hops),"-w",str(timeout),ip],
                                              stdout=subprocess.PIPE,text=True,shell=True))
        time.sleep(math.ceil(hops*(timeout/1000)*3.75*(1+amount*0.001)))
        outputs_temp=[]
        for process in processes:
            outputs_temp.append(process.stdout)
        output_cleaned=[]
        for output in outputs_temp:
            temp=[]
            for line in output:
                if line.strip()!='':
                    temp.append(line.strip())
            output_cleaned.append(temp)
        return output_cleaned
    
    def Analyze_Flow_Rate(self,pcap_data=stored_pcap_data,forced=False,verbose=True): #deprecated
        directory=self.stored_directory[0]
        filename='flow_rate.pkl'
        data_not_found=False
        if not forced: #load existing data if found
            if os.path.exists(f'{directory}/{filename}'): #if existing data is found
                self.logger.info(f'Found existing flow_rate data at {directory}/{filename}')
                read_list=self.Read_List(directory,filename)
                self.List_Initializer(self.stored_flow_rate,read_list)
                if verbose:
                    print(f'Found flow_rate data. [{len(read_list)}]')
                return read_list
            else:
                data_not_found=True
        if (forced or data_not_found):
            self.logger.info('Calculating flow rates')
            trans_time=self.stored_transmission_time[0]
            flow=self.Analyze_Flow(pcap_data)
            for ip_flow in flow:
                ip_flow[1]=math.trunc(float(ip_flow[1])/trans_time*100)/100
                ip_flow[2]=math.trunc(float(ip_flow[2])/trans_time*100)/100
            flow_rate=flow
            self.List_Initializer(self.stored_flow_rate,flow_rate)
            self.Write_List(flow_rate,self.stored_directory[0],'flow_rate')
        self.logger.debug('Stored flow rate in stored_flow_rate')
        return flow_rate
    def Query_Unique_IPs(self,unique_ips=stored_unique_ips,amount=20): #deprecated
        directory=self.stored_directory[0]
        if os.path.exists(f'{directory}/'):
            pass
        queried_unique_ips=[]
        to_be_queried=unique_ips.copy()
        
        self.logger.info(f'Querying unique IPs.')
        
        
        if len(self.stored_queried_ips)!=0: #if data already exists, then read it
            self.logger.debug(f'Found queried data. Pruning query list')
            for query in self.stored_queried_ips:   #prune the IPs to be queried
                for ip in to_be_queried:
                    try:
                        if query['query']==ip:
                            to_be_queried.remove(ip)
                    except KeyError:
                        pass
        if len(to_be_queried)!=0: #only if there are IPs to be queried, then query
            for ip in tqdm(to_be_queried[:min(amount,45)],desc='Querying IPs'):
                response=self.Query_IP(ip)
                queried_unique_ips.append(response)
                self.Write_JSON(response,self.stored_path[0][:self.stored_path[0].find('.')],response['query'])
        else:
            self.logger.info(f'All unique IPs are queried')
            print(f'All unique IPs are already queried')
            temp={'status':'Completed',
                  'amount':f'{len(self.stored_queried_ips)} IPs',
                  'time':f'{self.stored_transmission_time[0]} seconds'}
            self.Write_JSON(temp,self.stored_path[0][:self.stored_path[0].find('.')],'status')
        if len(self.stored_queried_ips)!=0: #if theres already data then append to it, else initialize it, if removed it will always overwrite data
            for query in queried_unique_ips:
                self.stored_queried_ips.append(query)
        else:
            self.List_Initializer(self.stored_queried_ips,queried_unique_ips)
        return queried_unique_ips
    def Generate_Summary(self,data=stored_queried_data): #deprecated
        self.logger.info(f'Summarizing data.')
        summarized=[]
        for query_data in data:
            temp={}
            query=query_data[1]
            if query['status']=='success':
                temp['Organization']=f"{query['as']}, {query['org']}, {query['isp']}"
                temp['Location']=f"{query['city']}, {query['regionName']}, {query['country']} | {query['lat']}, {query['lon']}"
                temp['IP']=f"{query['query']}"
            elif query['status']=='fail':
                temp['Location']=f'Unknown [Failed]'
                temp['Organization']=f'Unknown [Failed]'
                temp['IP']=f"{query['query']}"
            summarized.append(temp)
        self.logger.debug('Summarizing completed.')
        

In [ ]:
temp_features=['Burst Mixed', 'Avg Down PSize']
algos=['kmeans','agglomerative']
clusters=8

In [ ]:
A=Analyzer('PCAP Data/6400S-Dailies-Discord.pcapng',
           input=False,features=temp_features,
           query=True,forced_query=False,infinite_query=True,
           ping=False,cluster=False,clustering_algorithm='birch',clusters=clusters,samples=4,
           forced=False,verbose=True)

In [ ]:
A.Generate_Ensembling(features=temp_features,ensembling_algorithms=algos,show=False,clusters=clusters)

In [ ]:
A.Generate_Cluster(features=temp_features,clustering_algorithm=algos[0],clusters=clusters)

In [ ]:
A.Generate_Cluster(features=temp_features,clustering_algorithm=algos[1],clusters=clusters)

In [ ]:
A.Generate_Ensembling(features=temp_features,ensembling_algorithms=algos,clusters=clusters)

In [ ]:
A.stored_df[0].drop(['Destination Ports','Protocol(s)','First Pkt Time','Last Down Pkt Time','Last Up Pkt Time','Last Mixed Pkt Time','IAT Mixed','IAT Down','IAT Up'],axis=1).sort_values('Avg Down PSize')

In [ ]:
from sklearn.datasets import make_blobs

# Generate sample data
X, y_true = make_blobs(n_samples=100, centers=4, cluster_std=0.60, random_state=42)

# Apply KMeans
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

# Plot results
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
            s=200, c='red', marker='X')  # cluster centers
plt.title('K-Means Clustering')
plt.show()

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Sample 3D data
X, _ = make_blobs(n_samples=100, centers=4, n_features=3, random_state=42)

# Clustering
kmeans = KMeans(n_clusters=4)
labels = kmeans.fit_predict(X)

# Plotting in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=labels, cmap='viridis', s=50)
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Feature 3')
plt.title('3D Clustering')
plt.show()